Importamos las librerias necesarias

In [131]:
import pandas as pd
import numpy as np

Importamos los archivos como tablas

In [132]:
amazon = pd.read_csv('datasets/amazon_prime_titles.csv',dtype = {'show_id': str},delimiter = ',',encoding = "utf-8")
disney = pd.read_csv('datasets/disney_plus_titles.csv',dtype = {'show_id': str},delimiter = ',',encoding = "utf-8")
hulu = pd.read_csv('datasets/hulu_titles.csv',dtype = {'show_id': str},delimiter = ',',encoding = "utf-8")
netflix = pd.read_json('datasets/netflix_titles.json',encoding = "utf-8")

In [133]:
netflix.info()
disney.info()
hulu.info()
amazon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          

Se le añade la columna de plataforma

In [134]:
amazon["Plataforma"] = "Amazon"
netflix["Plataforma"] = "Netflix"
hulu["Plataforma"] = "Hulu"
disney["Plataforma"] = "Disney"

Concatenamos las tablas

In [135]:
frames = [amazon,disney,hulu,netflix]
movies = pd.concat(frames, ignore_index=True)

In [136]:
movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Plataforma
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon


Eliminamos las columnas que no necesitamos

In [110]:
movies = movies.drop("country", axis = 1)
movies = movies.drop("description", axis = 1)
movies = movies.drop("date_added", axis = 1)
movies = movies.drop("director", axis = 1)
movies = movies.drop("rating", axis = 1)
movies = movies.drop("show_id", axis = 1)


Normalizamos los datos faltantes


In [112]:
movies["cast"] = movies["cast"].replace(np.nan, "Sin Datos")
movies["duration"] = movies["duration"].replace(np.nan, 0)
movies["title"] = movies["title"].replace(np.nan, "Sin Datos")
movies = movies.fillna("Sin Datos")


Extraemos los valores de duration con la siguiente funcion

In [113]:
movies['duration'] = movies['duration'] .replace({'[a-zA-Z ]':''}, regex=True)

Normalizamos el tipo de dato


In [114]:
movies["duration"] = pd.to_numeric(movies["duration"])
movies["type"] = movies["type"].astype("string")
movies["title"] = movies["title"].astype("string")
movies["cast"] = movies["cast"].astype("string")
movies["Plataforma"] = movies["Plataforma"].astype("string")




Creamos una nueva columna con el tipo de dato de duracion ya sea min o season

In [115]:
movies["duration_type"] =  np.where(movies["type"] == "Movie", "min", "season")

Cambiamos nombres de columnas

In [116]:
movies.rename(columns= {"type": "Tipo", "title" : "Titulo", "cast": "Actores", "release_year" : "Anio", "duration" : "Duracion", "listed_in": "Genero", "duration_type" : "Duracion_tipo"} , inplace= True)

Vemos como queda el DataSet

In [117]:
movies.head()

,Tipo,Titulo,Actores,Anio,Duracion,Genero,Plataforma,Duracion_tipo
0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113,"Comedy, Drama",Amazon,min
1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110,"Drama, International",Amazon,min
2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74,"Action, Drama, Suspense",Amazon,min
3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69,Documentary,Amazon,min
4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45,"Drama, Fantasy",Amazon,min


Armamos la primera funcion: Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])

In [118]:
def get_max_duration(anio, plataforma, min_o_season):

    try:
        if min_o_season == "min":
            x = movies[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "Movie") & (movies["Plataforma"] == plataforma)]["Duracion"].max()
            y = (movies.loc[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "Movie") & (movies["Plataforma"] == plataforma) & (movies["Duracion"] == x)]["Titulo"]).item()
            return print(y, "tiene la mayor duracion: con ", x, min_o_season, "para el año", anio, "y la plataforma", plataforma) 

        if min_o_season == "season":
            x = movies[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "TV Show") & (movies["Plataforma"] == plataforma)]["Duracion"].max()
            y = (movies.loc[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "TV Show") & (movies["Plataforma"] == plataforma) & (movies["Duracion"] == x)]["Titulo"]).item()
            return print(y, "tiene la mayor duracion: con ", x, min_o_season, "para el año", anio, "y la plataforma", plataforma) 
        else:
            return print('Datos incorrectos o faltantes para su consulta, intente nuevamente')

    except ValueError:
        return print("La plataforma", plataforma, "no posee registros en", anio)
            

       
           
        
    

In [126]:
get_max_duration(1930, "Netflix", "season")

La plataforma Netflix no posee registros en 1930


Armamos la segunda funcion: Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)

In [120]:
def get_count_platform(plataforma):

    x =  movies[(movies["Tipo"] == "Movie") & (movies["Plataforma"] == plataforma )]["Titulo"].count()
    y =  movies[(movies["Tipo"] == "TV Show") & (movies["Plataforma"] == plataforma )]["Titulo"].count()

    return( print("La plataforma", plataforma, "tiene",x,"peliculas","y", y, "series"))

In [121]:
get_count_platform("Hulu")

La plataforma Hulu tiene 1484 peliculas y 1589 series


Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')

In [122]:
def get_listedin(genero):

    a = 0

    for i in movies[(movies["Plataforma"] == "Amazon")]["Genero"]: 
            
        x = list(i.split(","))

        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        
        if genero in lista:
            a+=1

    
    b = 0

    for i in movies[(movies["Plataforma"] == "Hulu")]["Genero"]: 
            
        
        x = list(i.split(","))
        
        
        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        


        if genero in lista:
            b+=1

    
    c = 0

    for i in movies[(movies["Plataforma"] == "Netflix")]["Genero"]: 
            
        
        x = list(i.split(","))
        
        
        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        


        if genero in lista:
            c+=1
    d = 0

    for i in movies[(movies["Plataforma"] == "Disney")]["Genero"]: 
            
        
        x = list(i.split(","))
        
        
        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        


        if genero in lista:
            d+=1
    return print("El Genero", genero,"aparece",a,"veces en Amazon,",b ,"veces en Hulu,", c, "veces en Netflix y", d, "veces en Disney")
    


In [123]:
get_listedin("Documentary")

El Genero Documentary aparece 993 veces en Amazon, 0 veces en Hulu, 0 veces en Netflix y 174 veces en Disney


Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)

In [124]:

def get_actor(plataforma, anio):

    lista = []


    for i in movies[(movies["Plataforma"] == plataforma) & (movies["Anio"] == anio)]["Actores"]:
        x = list(i.split(","))
        
        for n in x:
            lista.append(n.strip())



    actoresunicos = []
    numero=[]
    mayor = 0
    for n in lista:
        if n == "Sin Datos":
            continue
        if n  in actoresunicos:
            indice = actoresunicos.index(n)
            numero[indice] += 1
        
        if not n in actoresunicos:
            actoresunicos.append(n)
            numero.append(1)
       

    for valor in numero:
        if valor > mayor:
            mayor = valor
    actorunico = actoresunicos[numero.index(mayor)]               
    print("La actriz con mas apariciones para el año", anio, "en la plataforma", plataforma, "es", actorunico, "con", mayor, "apariciones" )
            
                
 

            

   

In [125]:
get_actor("Amazon", 2017)

La actriz con mas apariciones para el año 2017 en la plataforma Amazon es Maggie Binkley con 16 apariciones
